In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Tratamento dos dados e Estatística Descritiva
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
sns.set_style('whitegrid')

# Processamento de Texto
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import nltk.corpus   
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.preprocessing            import normalize

# Nuvem de palavras
from wordcloud                        import WordCloud

# K-Means
from sklearn.cluster import KMeans
from sklearn.metrics                  import silhouette_samples, silhouette_score, v_measure_score
from sklearn import cluster

# SVD
from sklearn.decomposition import TruncatedSVD

# Agglomerative Clustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

In [ ]:
#pasta de trabalho
dir = 'C:\\Users\\amand\\OneDrive\\Documentos\\TCC PUC\\NewsFake'

In [ ]:
files = glob.glob(dir + "/*.csv")

data_frame = pd.DataFrame()
content = []

for filename in files:
    
    # lendo conteúdo de um arquivo CSV
    df = pd.read_csv(filename, index_col=None)
    content.append(df)
  
# convertendo conteúdo para DataFrame
frame = pd.concat(content)
print(frame)

print('Datasets: \n   {}\n'.format(files))
print('Quantidade de Datasets:{}'.format(len(files)))

In [ ]:
dataset1 = pd.DataFrame()

frame[['https','vazio','fonte','resto']] = frame['news_url'].str.split('/',n=3,expand=True)
display(frame)

In [ ]:
dataset1 = frame[['title','type','fonte']].copy()

display(dataset1)

In [ ]:
#pasta de trabalho
dir2 = 'C:\\Users\\amand\\OneDrive\\Documentos\\TCC PUC\\ClaimFake'

In [ ]:
files2 = glob.glob(dir2 + "/*.csv")

data_frame2 = pd.DataFrame()
content2 = []

for filename in files2:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content2.append(df)
  
# convertendo conteúdo para data frame
frame2 = pd.concat(content2)
print(frame2)

print('Datasets: \n   {}\n'.format(files2))
print('Quantidade de Datasets:{}'.format(len(files2)))

In [ ]:
dataset2 = pd.DataFrame()

dataset2 = frame2[['title']].copy()
dataset2.loc[:, 'type'] = 'article'
dataset2.loc[:, 'fonte'] = 'www.medicalnewstoday.com'

display(dataset2)

In [ ]:
#pasta de trabalho
dir3 = 'C:\\Users\\amand\\OneDrive\\Documentos\\TCC PUC'

In [ ]:
files3 = glob.glob(dir3 + "/*.csv")

data_frame3 = pd.DataFrame()
content3 = []

for filename in files3:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content3.append(df)
  
# convertendo conteúdo em um dataframe
frame3 = pd.concat(content3)
print(frame3)

print('Datasets: \n   {}\n'.format(files3))
print('Quantidade de Datasets:{}'.format(len(files3)))

In [ ]:
dict = {'id': 'id',
        'tweet': 'title',
        'label': 'label'}

frame3.rename(columns=dict,
          inplace=True)

dataset3 = pd.DataFrame()

dataset3 = frame3[['title']].copy().loc[frame3['label'] == 'fake']
dataset3.loc[:, 'type'] = 'tweet'
dataset3.loc[:, 'fonte'] = 'twitter.com'

display(dataset3)

#conferindo se pegou todas as linhas "fake"

fake_frame = frame3.loc[frame3['label'] == 'fake']

print ('Quantidade de linhas com Fake News:{}'.format(len(fake_frame)))

In [ ]:
base_final = pd.DataFrame()
base_final = pd.concat([dataset1, dataset2, dataset3])

display(base_final)

base_final.loc[base_final.fonte == 'twitter.com','type'] = 'tweet'

display(base_final)

In [ ]:
base_final.loc[:, 'fonte'] = base_final['fonte'].apply(lambda x: 'Twitter' if x == 'twitter.com'
                                                       else 'Facebook' if x == 'www.facebook.com'
                                                       else 'Instagram' if x == 'www.instagram.com'
                                                       else 'Youtube' if x == 'www.youtube.com'
                                                       else 'Website')

display(base_final)

In [ ]:
base_final.duplicated().value_counts()

In [ ]:
#removendo valores duplicados
base_final.drop_duplicates(inplace=True)

base_final.duplicated().value_counts()

In [ ]:
base_final.loc[:, 'Tamanho_Texto'] = base_final['title'].apply(lambda x: len(x))

display(base_final)

In [ ]:
#separando hashtags
hash_frame = base_final['title'].tolist()
list_hashtags=[]
for text in hash_frame:
    for i in text.split():
        if i[0]=='#':
            list_hashtags.append(i)
            ''.join(list_hashtags)
            
hashtags = ','.join(list_hashtags)

df_hash = pd.DataFrame(list_hashtags, columns=['hashtag'])
display(df_hash)

In [ ]:
#Countplot do comparativo da quantidade total de tipo de "news"
fig = plt.subplots(figsize =(8,5))
sns.countplot(base_final['type'],palette='YlGnBu')
plt.title('Quantidade de tipos de posts', fontweight='bold', fontsize=16)
plt.xlabel('Tipo',fontweight='bold',fontsize=12)
plt.ylabel('Quantidade', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
#Countplot do comparativo da quantidade total de tipo de fonte
fig = plt.subplots(figsize =(8,5))
sns.countplot(base_final['fonte'],palette='YlGnBu')
plt.title('Quantidade de fontes', fontweight='bold', fontsize=16)
plt.xlabel('Fonte',fontweight='bold',fontsize=12)
plt.ylabel('Quantidade', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
# Plotting the pie chart for above dataframe
base_final.groupby(['type']).count().plot(
    kind='pie', y='title', autopct='%1.0f%%',
    colors = ['yellow','steelblue','orange'],
    title='Quantidade de textos por tipo')

In [ ]:
# Plotting the pie chart for above dataframe
media_frame = base_final.loc[base_final['type'] != 'tweet']
media_frame.groupby(['fonte']).count().plot(
    kind='pie', y='title', autopct='%1.0f%%',
    title='Proporção de outras redes sociais')

In [ ]:
#gráfico de distribuição tamanho dos titulos de noticias/posts/tweets
fig = plt.subplots(figsize=(7,3))
g = sns.distplot(base_final['Tamanho_Texto'])
plt.title('Distribuição de tamanhos de posts', fontweight='bold', fontsize=16)
plt.show()

In [ ]:
#gráfico de distribuição tamanho dos titulos de noticias/posts/tweets
fig = plt.subplots(figsize=(7,3))
g = sns.distplot(base_final['Tamanho_Texto'])
g.set(xlim=(0,750))
plt.title('Distribuição de tamanhos de posts', fontweight='bold', fontsize=16)
plt.show()

In [ ]:
#boxplot tamanho dos titulos de noticias/posts/tweets
fig,ax = plt.subplots(figsize=(8,5))
sns.boxplot(y=base_final['Tamanho_Texto'])
plt.title('Boxplot de tamanhos de posts', fontweight='bold', fontsize=16)
plt.show()

In [ ]:
base_final['Tamanho_Texto'].describe()

In [ ]:
base_final[base_final['Tamanho_Texto'] == 9]

In [ ]:
pd.options.display.max_colwidth = 4000
base_final[base_final['Tamanho_Texto'] == 8846]

In [ ]:
#Declarando funções para Stop Words e Stemming

# retira Stop Words
def tiraStopWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# aplica Stemming
def aplicaStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# retira palavras com menos de 2 caracteres e com mais de 21 caracteres
def tiraMinMaxCaracteres(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [ ]:
texto = base_final['title'].tolist()

def processaTexto(texto, lingua):   
    stopwords = nltk.corpus.stopwords.words(lingua)
    param_stemmer = SnowballStemmer(lingua)
    
    for title in texto:
        ind = texto.index(title)
        texto[ind] = texto[ind].replace('@', '') #removendo @
        texto[ind] = texto[ind].replace(',', '') #removendo ,
        texto[ind] = texto[ind].rstrip('\n') #removendo espaço em branco
        texto[ind] = texto[ind].casefold() #tornando todas as letras minúsculas

        texto[ind] = re.sub('\W_',' ', texto[ind]) #tirando caracteres especiais
        texto[ind] = re.sub("\S*\d\S*"," ", texto[ind]) #tira números e palavras concatenados com nº,ex: h4ck3r
        texto[ind] = re.sub("\S*@\S*\s?"," ", texto[ind]) #tira palavras com @
        texto[ind] = re.sub(r'http\S+', '', texto[ind]) #tira URL com http
        texto[ind] = re.sub(r'www\S+', '', texto[ind]) #tira URL com www
        
        listOfTokens = word_tokenize(texto[ind])
        minMaxWord = tiraMinMaxCaracteres(listOfTokens)
        
        listOfTokens = tiraStopWords(listOfTokens, stopwords)
        listOfTokens = tiraStopWords(listOfTokens, minMaxWord)
        listOfTokens = aplicaStemming(listOfTokens, param_stemmer)
        
        texto[ind]   = " ".join(listOfTokens)
        texto[ind] = unidecode(texto[ind])
    
    return texto

In [ ]:
lingua = 'english'
texto = processaTexto(texto, lingua)

display(texto)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texto)
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())

final_df = tf_idf
final_df

In [ ]:
conteudo = ' '.join(final_df)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(conteudo)
plt.figure(figsize=(16,9))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(hashtags)
plt.figure(figsize=(12,9))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# acha um número ótimo de componentes a ser utilizado para o Truncated SVD
n_comp = [4,10,15,20,50,100,150,200,500,700,800,900,1000,1500,2000,2500,3000,3500] 
explained = [] # variancia explicada para cada número de componente aplicado no Truncated SVD
for x in n_comp:
    svd = TruncatedSVD(n_components=x)
    svd.fit(final_df)
    explained.append(svd.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x,svd.explained_variance_ratio_.sum()))
plt.plot(n_comp, explained)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()

In [ ]:
svd = TruncatedSVD(n_components=1500)
final_df_svd = svd.fit_transform(final_df)

In [ ]:
final_df_svd

In [ ]:
def numero_k(mink, maxk, X):
  # Descobrindo qual o melhor valor de k
  wcss = []
  for i in range(mink,maxk):
    kmeans = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=200, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

  # Plot do gráfico do cotovelo para visualizar o melhor valor de k
  plt.figure(figsize=(10,5))
  plt.plot(range(mink,maxk), wcss, 'bx-')
  plt.xlabel('Valor de k')
  plt.ylabel('Inertia')
  plt.title('Método do Cotovelo exibindo número ótimo de k')
  plt.show()

  return

In [ ]:
numero_k(1, 20, final_df_svd)

In [ ]:
k = 10

for n_clusters in range(2 , k):
    clusterer = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = clusterer.fit_predict(final_df_svd)
    silhouette_avg = silhouette_score(final_df_svd, cluster_labels)
    print("Para n_clusters =", n_clusters,
          "O score_silhouette médio é :", silhouette_avg)

In [ ]:
k = 11
km = KMeans(algorithm='auto',
            copy_x=True,
            init='k-means++',
            max_iter=300,
            n_clusters=k,
            n_init=10,
            n_jobs=1,
            precompute_distances='auto',
            random_state=0,
            tol=0.0001,
            verbose=0)

In [ ]:
km.fit(final_df_svd)

In [ ]:
clusters = km.labels_
k_centers = km.cluster_centers_ 
original_space_centroids = svd.inverse_transform(k_centers)
order_centroids = original_space_centroids.argsort()[:, ::-1]

In [ ]:
svd_cluster_topics = {}
for c in range(k):
    topic = ','.join([centroids.columns[i] for i in [ix for ix in order_centroids[c, :10]]])
    svd_cluster_topics[c] = topic
    print ("Cluster %i: " % c + topic)

In [ ]:
# Inserindo a classificação de cluster para cada título
labels = km.labels_ 
base_final["Cluster_Kmeans_SVD"] = labels

In [ ]:
df_clusters = base_final["Cluster_Kmeans_SVD"].value_counts()
print(df_clusters)

In [ ]:
pd.options.display.max_colwidth = 193
base_final.groupby('Cluster_Kmeans_SVD').head(1).sort_values(by='Cluster_Kmeans_SVD')

In [ ]:
plt.figure(figsize=(12,9))
dendrograma = dendrogram(linkage(final_df_svd, method = 'ward'))
plt.title('Dendrograma')
plt.xlabel('Palavras')
plt.ylabel('Distância Euclidiana')

In [ ]:
hc = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage = 'ward')
previsoes = hc.fit_predict(final_df_svd)
labels = hc.labels_

In [ ]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices para cada cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # retorna score médio TF-IDF para cada palavra gerada no cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices com os top n_feats scores
        features = vectorizer.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
final_df_array = final_df.to_numpy()
n_feats = 20
dfs = get_top_features_cluster(final_df_array, previsoes, n_feats)
plotWords(dfs, 15)

In [ ]:
base_final["Cluster_Agglomerative"] = previsoes

df_clusters2 = base_final["Cluster_Agglomerative"].value_counts()
print(df_clusters2)

In [ ]:
pd.options.display.max_colwidth = 193
base_final.groupby('Cluster_Agglomerative').head(3).sort_values(by='Cluster_Agglomerative')

In [ ]:
fig = plt.subplots(figsize =(8,5))
sns.countplot(base_final['Cluster_Kmeans_SVD'],palette='YlGnBu')
plt.title('Quantidade de texto por Cluster', fontweight='bold', fontsize=16)
plt.xlabel('Cluster',fontweight='bold',fontsize=12)
plt.ylabel('Quantidade', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
fig = plt.subplots(figsize =(8,5))
sns.countplot(base_final['Cluster_Agglomerative'],palette='YlGnBu')
plt.title('Quantidade de texto por Cluster', fontweight='bold', fontsize=16)
plt.xlabel('Cluster',fontweight='bold',fontsize=12)
plt.ylabel('Quantidade', fontsize=12, fontweight='bold')
plt.show()